In [2]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# Tic-Tac-Toe

This notebook defines the game [tic-tac-toe](https://en.wikipedia.org/wiki/Tic-tac-toe).
It is played on a $3 \times 3$ board.  There are two players, which are called `X` and `O`.  Player `X` starts.  Player `X` always puts an `'X'` into an empty field on the board, while player `O` always puts an `'O'` in an empty field of the board.  The goal of the game for player `X` is to get three **Xs** into a row, column, or diagonal line, while player `O` needs to get three **Os** into a row, column, or diagonal line.

The global variable `gPlayers` returns a list with the name of both players.

In [3]:
gPlayers = [ 'X', 'O' ]

In the following naive implementation, states are represented as tuples of tuples of strings.  The game starts with an empty board.  An empty field on the board is represented by a blank space `' '`, while an `X` is represented by the string `'X'` and an `O` is represented as the string `'O'`.  The *start state* `gStart` defined below represents an empty board.

As we will later have need to store states in sets, we have to represent these states as 
tuples of tuples instead of lists of lists since lists are mutable and therefore can't be stored in sets.

In [2]:
gStart = tuple( tuple(' ' for col in range(3)) for row in range(3) )
gStart

((' ', ' ', ' '), (' ', ' ', ' '), (' ', ' ', ' '))

The function `to_list` transforms a tuple of tuples into a list of lists.  As tuples are immutable, this function is needed in order to create new states from given ones.

In [5]:
def to_list(State): 
    return [list(row) for row in State]

Conversely, the function `to_tuple` transforms a list of lists into a tuple of tuples.

In [6]:
def to_tuple(State): 
    return tuple(tuple(row) for row in State)

The function `toString(State)` represents the given state as a string.
This function is useful for debugging. 

In [7]:
def toString(State):
    result = '+-+-+-+\n'
    for Row in State:
        result += '|'
        for cell in Row:
            result += cell + '|'
        result += '\n+-+-+-+\n'
    return result

In [8]:
print(toString(gStart))

+-+-+-+
| | | |
+-+-+-+
| | | |
+-+-+-+
| | | |
+-+-+-+



Given a `State` and the `player` whose turn it is, the function `next_states(State, player)` computes the list of all states that can be reached from `State`.

In [9]:
def next_states(State, player):
    Result = []
    for row in range(3):
        for col in range(3):
            if State[row][col] == ' ':
                NextState           = to_list(State)
                NextState[row][col] = player
                NextState           = to_tuple(NextState)
                Result.append(NextState)
    return Result

What are the possible moves in the start state?

In [10]:
for S in next_states(gStart, 'X'):
    print(toString(S))

+-+-+-+
|X| | |
+-+-+-+
| | | |
+-+-+-+
| | | |
+-+-+-+

+-+-+-+
| |X| |
+-+-+-+
| | | |
+-+-+-+
| | | |
+-+-+-+

+-+-+-+
| | |X|
+-+-+-+
| | | |
+-+-+-+
| | | |
+-+-+-+

+-+-+-+
| | | |
+-+-+-+
|X| | |
+-+-+-+
| | | |
+-+-+-+

+-+-+-+
| | | |
+-+-+-+
| |X| |
+-+-+-+
| | | |
+-+-+-+

+-+-+-+
| | | |
+-+-+-+
| | |X|
+-+-+-+
| | | |
+-+-+-+

+-+-+-+
| | | |
+-+-+-+
| | | |
+-+-+-+
|X| | |
+-+-+-+

+-+-+-+
| | | |
+-+-+-+
| | | |
+-+-+-+
| |X| |
+-+-+-+

+-+-+-+
| | | |
+-+-+-+
| | | |
+-+-+-+
| | |X|
+-+-+-+



The variable `gAllLines` collects the coordinates of all the fields of the three *rows*, the three *columns*, and the two *diagonals*.  This variable is needed to check whether the game has already terminated.

The name of the variable is prefixed with `g` to distinguish the variable as a *global variable*.

In [11]:
gAllLines = [ [ (row, col) for col in range(3) ] for row in range(3) ] \
          + [ [ (row, col) for row in range(3) ] for col in range(3) ] \
          + [ [ (0, 0), (1, 1), (2, 2) ] ]                             \
          + [ [ (0, 2), (1, 1), (2, 0) ] ]
gAllLines

[[(0, 0), (0, 1), (0, 2)],
 [(1, 0), (1, 1), (1, 2)],
 [(2, 0), (2, 1), (2, 2)],
 [(0, 0), (1, 0), (2, 0)],
 [(0, 1), (1, 1), (2, 1)],
 [(0, 2), (1, 2), (2, 2)],
 [(0, 0), (1, 1), (2, 2)],
 [(0, 2), (1, 1), (2, 0)]]

The function `utility` takes one argument:
- `State` is a tuple of tuples representing the board.
 
The function returns `1` if the player `X` has won the game, `-1` if the game is lost for player `X`, `0` if it's a draw, and `None` if the game has not yet been decided.

In [12]:
def utility(State):
    for Line in gAllLines:
        Marks = { State[row][col] for row, col in Line }
        if len(Marks) == 1 and  Marks != { ' ' }: 
            if Marks == { 'X' }:
                return  1
            else:
                return -1
    for row in range(3):
        for col in range(3):
            if State[row][col] == ' ':
                return None  # the board is not filled  
    # at this point, the board has been filled, but there is no winner, hence it's a draw
    return 0            

In [13]:
print(utility(gStart))

None


`finished(State)` is `True` if and only if the game is over and hence the function `utility(State)` returns a value different from `None`.

In [14]:
def finished(State): 
    return utility(State) != None

In [15]:
finished(gStart)

False

The function `get_move` asks the user to input a move in the format `r,c` where `r` is the row and the `c` is the column where the next symbol is to be placed. Rows and columns are counted starting from `0`.  It is assumed that the user plays as `'O'`, i.e. the user takes the second move.

In [16]:
def get_move(State):
    State = to_list(State)
    while True:
        try:
            row, col = input('Enter move here: ').split(',')
            row, col = int(row), int(col)
            if State[row][col] == ' ':
                State[row][col] = 'O'
                return to_tuple(State)
            print("Don't cheat! Please try again.")  
        except:
            print('Illegal input.')  
            print('row and col are numbers from the set {0,1,2}.')

This function informs the player, who is assumed to be `'O'`, about the result of the game once the game is finished.

In [17]:
def final_msg(State):
    if finished(State):
        if utility(State) == -1:
            print('You have won!')
        elif utility(State) == 1:
            print('The computer has won!')
        else:
            print("It's a draw.")
        return True
    return False

# Drawing the Board

In [18]:
import ipycanvas as cnv

In [19]:
g_size = 150

This function creates the canvas for the start state.  It draws an empty board which is later used for the game.

In [20]:
def create_canvas():
    n = 3
    canvas = cnv.Canvas(size=(g_size * n, g_size * n + 50))
    display(canvas)
    return canvas

This function takes three arguments:
- `State` is the current state of the game.
- `canvas` is a canvas used to draw the state.
- `value` is the value of the game for player `X`.

The function draws the given `State` onto `canvas`.  Below that, the `value` is printed.

In [21]:
def draw(State, canvas, value):
    canvas.clear()
    n = len(State)
    canvas.font = '90px sans-serif'
    canvas.text_align    = 'center'
    canvas.text_baseline = 'middle'
    for row in range(n):
        for col in range(n):
            x = col * g_size
            y = row * g_size
            canvas.line_width = 3.0
            canvas.stroke_rect(x, y, g_size, g_size)
            symbol = State[row][col]
            if symbol != ' ':
                x += g_size // 2
                y += g_size // 2
                if symbol == 'X':
                    canvas.fill_style ='red'
                else:
                    canvas.fill_style ='blue'
                canvas.fill_text(symbol, x, y)
    canvas.font = '12px sans-serif'
    canvas.fill_style = 'green'
    for row in range(n):
        for col in range(n):
            x = col * g_size + 16
            y = row * g_size + 141
            canvas.fill_text(f'({row}, {col})', x, y)            
    canvas.font = '20px sans-serif'
    canvas.fill_style = 'black'
    x = 1.5 * g_size
    y = 3.2 * g_size
    canvas.fill_text(str(value), x, y)

In [22]:
State = (('O', 'X', 'O'),
         ('X', 'O', 'X'),
         ('O', 'X', 'O')
        )
         
draw(State, create_canvas(), 1)

Canvas()

## Checking the Memory Footprint

In [23]:
gStart

((' ', ' ', ' '), (' ', ' ', ' '), (' ', ' ', ' '))

In [1]:
import sys

The memory needed to store a single state is the memory needed to store a list containing three objects  plus the memory needed to store those objects.

In [3]:
sys.getsizeof(gStart) + sum([sys.getsizeof(row) for row in gStart]) + 9 * sys.getsizeof('_')

706

To compare, a single positive integer less than $1,073,741,824$ needs 28 bytes.

In [26]:
sys.getsizeof(1073741823)

28